# now by using yolo5s we are able to detect only the vehicals (no labling and recording )

In [ ]:
import cv2
import numpy as np
import torch
from filterpy.kalman import KalmanFilter

# Load the pre-trained YOLOv5 model from PyTorch Hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # 'yolov5s' is a lightweight model

# SORT Tracker Class
class SORT:
    def __init__(self):
        self.trackers = []
        self.next_id = 0

    def update(self, detections):
        updated_tracks = []
        for det in detections:
            # Initialize Kalman filter for each new detection
            tracker = KalmanFilter(dim_x=7, dim_z=4)
            tracker.x[:4] = np.array([det[0], det[1], det[2], det[3]]).reshape((4, 1))
            tracker.F = np.eye(7)  # State transition matrix
            tracker.H = np.eye(4, 7)  # Measurement function
            tracker.P *= 10.

            # Assign unique IDs and add to trackers list
            tracker.id = self.next_id
            self.next_id += 1
            updated_tracks.append(tracker)

        return updated_tracks

# Capture video from webcam (default camera index is 0)
cap = cv2.VideoCapture(0)  # Change to 1 or another index if you have multiple cameras

# Initialize SORT tracker
tracker = SORT()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Object detection using YOLOv5
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()  # Bounding boxes (x1, y1, x2, y2, conf, class)

    # Filter detections for vehicles (car, truck, bus, etc.)
    vehicle_classes = [2, 3, 5, 7]  # COCO classes: 2=car, 3=motorbike, 5=bus, 7=truck
    vehicles = [d for d in detections if int(d[5]) in vehicle_classes and d[4] > 0.4]  # Confidence threshold

    # Update SORT tracker with vehicle detections
    tracks = tracker.update(vehicles)

    # Draw Bounding Boxes and Track IDs
    for track in tracks:
        x1, y1, x2, y2 = track.x[:4].flatten()
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track.id}', (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Display the frame with tracking
    cv2.imshow('Real-Time Multi-Object Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
